In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, make_scorer, classification_report
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
import mord
from collections import Counter

Using TensorFlow backend.


In [2]:
path = "/Users/Stijn/Documents/Master Data Science and Society/Block 3/thesis/code/thesis_Code/"
mood = pd.read_csv(path+'mood_imputed_median.csv', sep = ',', index_col=0)
mood['content_last'] = mood.groupby('user_id')['content'].shift()
mood.loc[(pd.isnull(mood.content_last)), 'content_last'] = mood['content']
mood_content = mood.drop(["anxious", "bored", "cheerful", "user_id", "response_time"], axis=1)

In [3]:
mood_content[:5]

,content,day_time_window,average_TimeUse,bulk,messaging,socialnetworking,otherapp,content_last
4825,4.0,3.0,0.673000,0,0,0,1,4.0
4826,4.0,4.0,64.314667,0,1,0,2,4.0
245,4.0,1.0,20.590500,0,0,0,2,4.0
4827,4.0,2.0,0.000000,0,0,0,0,4.0
4828,4.0,3.0,53.957200,0,3,0,2,4.0


In [4]:
# Normalize predictors
mood_normz = mood_content.copy()
features_to_normalize = ['day_time_window', 'average_TimeUse', 'messaging', 'socialnetworking', 'otherapp']
mood_normz[features_to_normalize] = mood_normz[features_to_normalize].apply(lambda x:(x-x.min()) / (x.max()-x.min()))

In [5]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [6]:
# Oversampling unbalanced target
print("Before OverSampling, counts of labels: {}".format(y['content'].value_counts()))
  
sm = SMOTENC(categorical_features=[0, 2, 6], random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X, y['content'].ravel()) 

print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0))) 
print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1))) 
print("After OverSampling, counts of label '2': {}".format(sum(y_train_res == 2))) 
print("After OverSampling, counts of label '3': {}".format(sum(y_train_res == 3))) 
print("After OverSampling, counts of label '4': {}".format(sum(y_train_res == 4))) 
print("After OverSampling, counts of label '5': {}".format(sum(y_train_res == 5))) 

Before OverSampling, counts of labels: 3    3972
4    2958
2    2775
1    1559
0    1090
5     867
Name: content, dtype: int64
After OverSampling, counts of label '0': 3972
After OverSampling, counts of label '1': 3972
After OverSampling, counts of label '2': 3972
After OverSampling, counts of label '3': 3972
After OverSampling, counts of label '4': 3972
After OverSampling, counts of label '5': 3972


In [7]:
# Convert oversample dataset to Pandas DataFrame
X_train = pd.DataFrame(data=X_train_res)
y_train = pd.DataFrame(data=y_train_res)

In [8]:
X_train_nomood = X_train.iloc[:,:-1]
X_train_onlyboredlast = X_train.iloc[:,-1:]

In [9]:
X_train = X_train_nomood.values
y_train = y_train.values.ravel()

In [10]:
# Shuffle both train and test set to cancel patterns
X, y = shuffle(X_train, y_train)

In [11]:
rfc=RandomForestClassifier(random_state=42)
parameter_candidates = { 
    'n_estimators': [100, 200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}
clf = GridSearchCV(estimator=rfc, 
                    param_grid=parameter_candidates,
                    cv=10,
                    n_jobs=-1)

In [12]:
clf_fit = clf.fit(X, y)

In [13]:
print('Best score:', clf_fit.best_score_) 
print(" ")
print('Best estimator:',clf_fit.best_estimator_) 

Best score: 0.2756797583081571
 
Best estimator: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [5]:
def macroaverage_mae(test, pred):
    mae_0, mae_1, mae_2, mae_3, mae_4, mae_5 = (0,0,0,0,0,0)

    for i in range(len(pred)):
        if test[i] == 0 and pred[i] != 0:
            mae_0 += (abs(test[i]-pred[i]))
        if test[i] == 1 and pred[i] != 1:
            mae_1 += (abs(test[i]-pred[i]))
        if test[i] == 2 and pred[i] != 2:
            mae_2 += (abs(test[i]-pred[i]))
        if test[i] == 3 and pred[i] != 3:
            mae_3 += (abs(test[i]-pred[i]))
        if test[i] == 4 and pred[i] != 4:
            mae_4 += (abs(test[i]-pred[i]))
        if test[i] == 5 and pred[i] != 5:
            mae_5 += (abs(test[i]-pred[i]))

    cnt_0, cnt_1, cnt_2, cnt_3, cnt_4, cnt_5 = (0,0,0,0,0,0)
    for z in range(len(test)):
        if test[z] == 0:
            cnt_0 += 1
        if test[z] == 1:
            cnt_1 += 1
        if test[z] == 2:
            cnt_2 += 1
        if test[z] == 3:
            cnt_3 += 1
        if test[z] == 4:
            cnt_4 += 1
        if test[z] == 5:
            cnt_5 += 1

    mae_macroaverage = ((mae_0/cnt_0) + (mae_1/cnt_1) + (mae_2/cnt_2) + (mae_3/cnt_3) + (mae_4/cnt_4) + (mae_5/cnt_5)) / 6
    return mae_macroaverage 

#score = make_scorer(macroaverage_mae, greater_is_better=False)

In [6]:
# X = predictors, y = target
X = mood_normz.iloc[:,1:-1]
y = mood_normz.iloc[:, 0:1]

# Convert float to int
y = y.astype(int)

In [7]:
X_train, y_train = shuffle(X, y)

In [8]:
# Convert oversample dataset to Pandas DataFrame
X = pd.DataFrame(data=X_train)
y = pd.DataFrame(data=y_train)

In [9]:
n_split=10
f1_scores = []
f1_scores_classes = []
mmae_scores = []
for train_index, test_index in KFold(n_splits = n_split, random_state=2, shuffle=True).split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    sm = SMOTENC(categorical_features=[0, 2], random_state = 2) 
    X_train_oversampl, y_train_oversampl = sm.fit_sample(X_train, y_train['content'].ravel()) 
    X_use, y_use = shuffle(X_train_oversampl, y_train_oversampl)
    
    model = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
                                       
    model.fit(X_use, y_use )                                    
    pred = model.predict(X_test)
    print("Predicted values: ", Counter(pred))
    
    f1_scores.append(round(f1_score(y_test, pred, average='weighted'), 4))
    f1_scores_classes.append(f1_score(y_test, pred, average=None))
    
    test = y_test.values
    mmae_scores.append(macroaverage_mae(test, pred))

Predicted values:  Counter({1: 638, 5: 170, 3: 168, 0: 142, 4: 105, 2: 100})
Predicted values:  Counter({1: 457, 5: 337, 3: 225, 0: 153, 4: 104, 2: 46})
Predicted values:  Counter({1: 639, 3: 199, 5: 173, 0: 147, 2: 100, 4: 64})
Predicted values:  Counter({1: 612, 3: 181, 5: 181, 0: 173, 4: 90, 2: 85})
Predicted values:  Counter({1: 624, 3: 215, 0: 161, 5: 152, 2: 86, 4: 84})
Predicted values:  Counter({1: 606, 3: 206, 0: 171, 5: 144, 4: 111, 2: 84})
Predicted values:  Counter({1: 659, 3: 188, 0: 161, 5: 144, 4: 93, 2: 77})
Predicted values:  Counter({1: 616, 3: 220, 0: 150, 5: 139, 4: 136, 2: 61})
Predicted values:  Counter({1: 451, 5: 323, 3: 205, 0: 180, 4: 96, 2: 67})
Predicted values:  Counter({1: 502, 5: 294, 3: 240, 0: 135, 2: 87, 4: 64})


In [10]:
print("Mean F1-score per class: ", [round(np.mean(x), 4) for x in zip(*f1_scores_classes)])
print("STD F1-score per class: ", [round(np.std(x), 4) for x in zip(*f1_scores_classes)])

Mean F1-score per class:  [0.1948, 0.2107, 0.1015, 0.2376, 0.1303, 0.1298]
STD F1-score per class:  [0.0296, 0.0186, 0.0214, 0.0216, 0.0191, 0.0202]


In [11]:
print("Macro-averaged F1-scores: ", f1_scores)
print("Mean MA F1-scores: ", round(np.mean(f1_scores), 4))
print("Std MA F1-scores: ", round(np.std(f1_scores), 4))

Macro-averaged F1-scores:  [0.1738, 0.168, 0.1628, 0.1532, 0.1758, 0.1868, 0.1811, 0.1764, 0.1734, 0.1633]
Mean MA F1-scores:  0.1715
Std MA F1-scores:  0.0093


In [12]:
print("Macro-averaged MAE: ", mmae_scores)
print("Mean MMAE-scores: ", round(np.mean(mmae_scores), 4))
print("Std MMAE-scores: ", round(np.std(mmae_scores), 4))

Macro-averaged MAE:  [array([1.76935538]), array([1.95363351]), array([1.72667533]), array([1.81793937]), array([1.79240356]), array([1.73019786]), array([1.79226374]), array([1.74769455]), array([1.88252694]), array([1.87692796])]
Mean MMAE-scores:  1.809
Std MMAE-scores:  0.0706
